<a href="https://colab.research.google.com/github/stephen-njiu/Ai_Recommendation_Systems/blob/main/Movie_recommendation_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
len(movies), len(ratings)

(9742, 100836)

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies = pd.read_csv("final_movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
len(movies)

86537

In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


<b>Defining the parser to read the data into surprise dataframe. The parser requires the scale or rating and the columns in a specific order ['userid, 'movieid', 'rating']

In [7]:
# !pip install scikit-surprise

In [8]:
import surprise

In [9]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [10]:
# Define the data (ratings attributes)
data = ratings[['userId', 'movieId', 'rating']]
data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [11]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
from surprise import SVD
from surprise import dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import KNNWithMeans

In [12]:
no_of_users = 600
reader = Reader(rating_scale=(-10,0))

In [13]:
class MyDataset(dataset.DatasetAutoFolds):
  def __init__(self, data, reader):
    self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in data.values]
    self.reader = reader

final_data = MyDataset(data, reader)

In [14]:
final_data

### Define the Simulation Parameters

Algorithm Type

User_Based vs Item_based

Similarity Metric

In [15]:
sim_parameters = {'name': 'cosine', 'user_based':False}
algo = KNNWithMeans(sim_options=sim_parameters)

Cross Validation Accuracies

In [16]:
algo = SVD()
cross_validate(algo, final_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.6546  3.6505  3.6454  3.6713  3.6453  3.6534  0.0096  
MAE (testset)     3.5047  3.4993  3.4911  3.5189  3.4937  3.5016  0.0099  
Fit time          8.81    4.87    1.85    1.59    1.66    3.76    2.81    
Test time         1.12    0.12    0.22    0.12    0.21    0.36    0.38    


{'test_rmse': array([3.65460612, 3.65049849, 3.64544709, 3.6713336 , 3.64534337]),
 'test_mae': array([3.50473522, 3.4993058 , 3.49114891, 3.51894184, 3.493653  ]),
 'fit_time': (8.81013560295105,
  4.872943639755249,
  1.8544814586639404,
  1.5857181549072266,
  1.658055305480957),
 'test_time': (1.116166114807129,
  0.12497925758361816,
  0.2171480655670166,
  0.11504554748535156,
  0.21451616287231445)}

### Train the model on Complete dataset

In [17]:
trainset = final_data.build_full_trainset()
algo.fit(trainset)

In [18]:
testset = trainset.build_anti_testset()


Make Predictions

In [19]:
predictions = algo.test(testset)

In [20]:
predictions

[Prediction(uid=1.0, iid=318.0, r_ui=3.501556983616962, est=0, details={'was_impossible': False}),
 Prediction(uid=1.0, iid=1704.0, r_ui=3.501556983616962, est=0, details={'was_impossible': False}),
 Prediction(uid=1.0, iid=6874.0, r_ui=3.501556983616962, est=0, details={'was_impossible': False}),
 Prediction(uid=1.0, iid=8798.0, r_ui=3.501556983616962, est=0, details={'was_impossible': False}),
 Prediction(uid=1.0, iid=46970.0, r_ui=3.501556983616962, est=0, details={'was_impossible': False}),
 Prediction(uid=1.0, iid=48516.0, r_ui=3.501556983616962, est=0, details={'was_impossible': False}),
 Prediction(uid=1.0, iid=58559.0, r_ui=3.501556983616962, est=0, details={'was_impossible': False}),
 Prediction(uid=1.0, iid=60756.0, r_ui=3.501556983616962, est=0, details={'was_impossible': False}),
 Prediction(uid=1.0, iid=68157.0, r_ui=3.501556983616962, est=0, details={'was_impossible': False}),
 Prediction(uid=1.0, iid=71535.0, r_ui=3.501556983616962, est=0, details={'was_impossible': Fals

## Function for Prediction
A function to make the top 5 predictions for each user

In [21]:
# Fetching the top five predictions for the user
from collections import defaultdict
def get_top(predictions, n=5):
  top_n = defaultdict(list)
  ''''returns the top n predictions for each user from a set of predictions
  Args:
    predictions(list of Prediction objects): The list of predictions, as
      returned by the test method of an algorithm.
    n(int): The number of recommendation to output for each user. Default
      is 5.
  Returns:
  A dict where keys are user (raw) ids and values are lists of tuples:
      [(raw item id, rating estimation), ...] of size n.

  '''

  # First map predictions to each user
  top_n = defaultdict(list)
  for uid, iid, true_r, est, _ in predictions:
    top_n[uid].append((iid, est))
  # then sort the predictions for each user and retrieve the highest score
  for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:n]
  return top_n


In [22]:
top_n = get_top(predictions, n=5)

In [23]:
top_n

defaultdict(list,
            {1.0: [(318.0, 0),
              (1704.0, 0),
              (6874.0, 0),
              (8798.0, 0),
              (46970.0, 0)],
             2.0: [(1.0, 0), (3.0, 0), (6.0, 0), (47.0, 0), (50.0, 0)],
             3.0: [(1.0, 0), (3.0, 0), (6.0, 0), (47.0, 0), (50.0, 0)],
             4.0: [(1.0, 0), (3.0, 0), (6.0, 0), (50.0, 0), (70.0, 0)],
             5.0: [(3.0, 0), (6.0, 0), (47.0, 0), (70.0, 0), (101.0, 0)],
             6.0: [(1.0, 0), (70.0, 0), (101.0, 0), (157.0, 0), (223.0, 0)],
             7.0: [(3.0, 0), (6.0, 0), (47.0, 0), (70.0, 0), (101.0, 0)],
             8.0: [(1.0, 0), (3.0, 0), (6.0, 0), (70.0, 0), (101.0, 0)],
             9.0: [(1.0, 0), (3.0, 0), (6.0, 0), (47.0, 0), (50.0, 0)],
             10.0: [(1.0, 0), (3.0, 0), (6.0, 0), (47.0, 0), (50.0, 0)],
             11.0: [(1.0, 0), (3.0, 0), (47.0, 0), (50.0, 0), (70.0, 0)],
             12.0: [(1.0, 0), (3.0, 0), (6.0, 0), (47.0, 0), (50.0, 0)],
             13.0: [(1.0, 0), (3.0,

#Function to get Users and Corresponding Predictions

In [24]:
from itertools import islice

def take(n, iterable):
  "Return first n items of the iterable as a list"
  return list(islice(iterable, n))

## Prediction Matrix

In [25]:
for uid, user_ratings in take(5, top_n.items()):
  print(uid, [iid for (iid, _) in user_ratings])
  print("for User ",uid)
  for (iid, rating) in user_ratings:
    if iid > len(movies):
      iid = np.random.randint(1, len(movies))
    print(iid)
    print(movies.loc[int(iid), 'title'])

1.0 [318.0, 1704.0, 6874.0, 8798.0, 46970.0]
for User  1.0
318.0
Swimming with Sharks (1995)
1704.0
Further Gesture, A (1996)
6874.0
House of Cards (1993)
8798.0
Grapes of Death, The (Raisins de la mort, Les) (1978)
46970.0
A la mala (2015)
2.0 [1.0, 3.0, 6.0, 47.0, 50.0]
for User  2.0
1.0
Jumanji (1995)
3.0
Waiting to Exhale (1995)
6.0
Sabrina (1995)
47.0
Pocahontas (1995)
50.0
Guardian Angel (1994)
3.0 [1.0, 3.0, 6.0, 47.0, 50.0]
for User  3.0
1.0
Jumanji (1995)
3.0
Waiting to Exhale (1995)
6.0
Sabrina (1995)
47.0
Pocahontas (1995)
50.0
Guardian Angel (1994)
4.0 [1.0, 3.0, 6.0, 50.0, 70.0]
for User  4.0
1.0
Jumanji (1995)
3.0
Waiting to Exhale (1995)
6.0
Sabrina (1995)
50.0
Guardian Angel (1994)
70.0
Fair Game (1995)
5.0 [3.0, 6.0, 47.0, 70.0, 101.0]
for User  5.0
3.0
Waiting to Exhale (1995)
6.0
Sabrina (1995)
47.0
Pocahontas (1995)
70.0
Fair Game (1995)
101.0
Unforgettable (1996)


In [26]:
#printing top prediction matrix
for uid, user_ratings in take(5, top_n.items()):
  print("for User ",uid)
  for (iid, rating) in user_ratings:
    if iid > len(movies):
      iid = np.random.randint(1, len(movies))
    print(iid)
    print(movies.loc[int(iid), 'title'])

for User  1.0
318.0
Swimming with Sharks (1995)
1704.0
Further Gesture, A (1996)
6874.0
House of Cards (1993)
8798.0
Grapes of Death, The (Raisins de la mort, Les) (1978)
46970.0
A la mala (2015)
for User  2.0
1.0
Jumanji (1995)
3.0
Waiting to Exhale (1995)
6.0
Sabrina (1995)
47.0
Pocahontas (1995)
50.0
Guardian Angel (1994)
for User  3.0
1.0
Jumanji (1995)
3.0
Waiting to Exhale (1995)
6.0
Sabrina (1995)
47.0
Pocahontas (1995)
50.0
Guardian Angel (1994)
for User  4.0
1.0
Jumanji (1995)
3.0
Waiting to Exhale (1995)
6.0
Sabrina (1995)
50.0
Guardian Angel (1994)
70.0
Fair Game (1995)
for User  5.0
3.0
Waiting to Exhale (1995)
6.0
Sabrina (1995)
47.0
Pocahontas (1995)
70.0
Fair Game (1995)
101.0
Unforgettable (1996)
